In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.linear_model import LinearRegression
import warnings
from pandas.tseries.offsets import DateOffset
warnings.filterwarnings('ignore')
import seaborn as sn
from tqdm import tqdm_notebook

In [12]:
#Define function for inserting data
def insert_data(data,data2,unique_vals,result_list):
    for i in tqdm_notebook(range(len(unique_vals))):
        temp = data.loc[data['tic'] == unique_vals[i]] ; temp = temp[~temp.index.duplicated(keep='last')]
        temp2 = data2.loc[data2['tic'] == unique_vals[i]] ; temp2 = temp2[~temp2.index.duplicated(keep='last')]
        if not temp.empty and not temp2.empty:
            result_list[i] = pd.concat([result_list[i], temp])
            result_list[i] = pd.concat([result_list[i], temp2],axis=1)
            result_list[i] = result_list[i].reset_index()
            result_list[i] = result_list[i].fillna(method = "ffill")
    return result_list

In [13]:
#Define dictionary for results --> Convert into list and assign some data to the dataframes
def dic_func(unique_vals):
    c = {}
    for i in range(len(unique_vals)):
        c["corp_{0}".format(i)] = []
    result_list = list(c.items())

    for i in range(len(unique_vals)):
        result_list[i] = pd.DataFrame()                                                     

    for i in range(len(unique_vals)):
        result_list[i] = pd.DataFrame(result_list[i])
    return result_list

In [14]:
def unique_vals_func(df):
    unique_vals = np.unique(df['tic'])
    return unique_vals 

# Dividend-to-price ratio

In [17]:
def DTOP_func(unique_vals,result_list):
    for i in range(len(unique_vals)):
        result_list[i]["DTOP"] = np.NaN
        if len(result_list[i])>1:
            result_list[i]["DTOP"] = result_list[i]["dvpsxm"].shift(1)/result_list[i]["prccm"]
    return result_list

# Earnings Quality

### Accruals - balance sheet

In [18]:
def ABS_func(unique_vals,result_list):
    for i in range(len(unique_vals)):
        result_list[i]["ABS"] = np.NaN
        if len(result_list[i])>1:
            result_list[i]["ABS"] = ((((result_list[i]["actq"]-result_list[i]["actq"].shift(1))-(result_list[i]["chq"]-result_list[i]["chq"].shift(1)))) - ((result_list[i]["lctq"]-result_list[i]["lctq"].shift(1))-(result_list[i]["dlcq"]-result_list[i]["dlcq"].shift(1)))-result_list[i]["dpq"])/result_list[i]["atq"]
    return result_list

### Variability in sales

In [19]:
def VSAL_func(unique_vals,result_list):
    for i in range(len(unique_vals)):
        result_list[i]["VSAL"] = np.NaN
        if len(result_list[i])>19:
            for j in reversed(range(len(result_list[i])-19)):
                result_list[i].at[j+19,"VSAL"] = np.std(result_list[i]["saleq"].iloc[j:j+20])
    return result_list


# Leverage

### DTOA

In [22]:
def DTOA_func(unique_vals,result_list):
    for i in range(len(unique_vals)):
        result_list[i]["DTOA"] = (result_list[i]["ltq"]+result_list[i]["dlttq"])/result_list[i]["atq"]
    return result_list

# Growth

### Egro

In [23]:
def EGRO_func(unique_vals,result_list):
    for i in range(len(result_list)):
        result_list[i]["EGRO"] = np.NaN
        if len(result_list[i])>19:
            result_list[i] = result_list[i].sort_index(ascending=False)
            #Make 5 years timeline for regression
            x = np.array([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]).reshape((-1, 1))
            temp_reg = np.zeros((len(result_list[i])))
            temp_sum = np.zeros((len(result_list[i])))
            k=0
            for j in range(len(result_list[i])-19):
                try:
                    temp_sum[j] = (result_list[i]["epspxq"][k:k+20].sum())/20
                    if temp_sum[j]>0:
                        temp = result_list[i]["epspxq"][k:k+20].to_numpy().reshape((-1, 1))
                        model = LinearRegression().fit(x, temp)
                        temp_reg[j] = model.coef_*(-1)
                    else:
                        temp_sum[j] = np.NaN
                        temp_reg[j] = np.NaN
                    k = k+1
                except:
                    temp_sum[j] = np.NaN
                    temp_reg[j] = np.NaN
                    k = k+1
       
            result_list[i]["EGRO"] = temp_reg/temp_sum
            result_list[i] = result_list[i].sort_index(ascending=True)
        
    return result_list

### SGRO

In [24]:
def SGRO_func(unique_vals,result_list):
    for i in range(len(result_list)):
        result_list[i]["sales per share"] = result_list[i]["saleq"] / result_list[i]["cshoq"]
        result_list[i]["SGRO"] = np.NaN
        if len(result_list[i])>19:
            result_list[i] = result_list[i].sort_index(ascending=False)
            #Make 5 years timeline for regression
            x = np.array([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]).reshape((-1, 1))
            temp_reg = np.zeros((len(result_list[i])))
            temp_sum = np.zeros((len(result_list[i])))
            k=0
            for j in range(len(result_list[i])-19):
                try:
                    temp = result_list[i]["sales per share"][k:k+20].to_numpy().reshape((-1, 1))
                    model = LinearRegression().fit(x, temp)
                    temp_reg[j] = model.coef_*(-1)                
                    temp_sum[j] = (result_list[i]["sales per share"][k:k+20].sum())/20
                    k = k+1

                except:
                    temp_sum[j] = np.NaN
                    temp_reg[j] = np.NaN
                    k = k+1

            result_list[i]["SGRO"] = temp_reg/temp_sum
            result_list[i] = result_list[i].sort_index(ascending=True)
    return result_list

# Liquidity (Both monthly)

### Montly share turnover

### Quarterly turnover 

In [25]:
def STOM_func(unique_vals,result_list):
    for i in range(len(unique_vals)):
        result_list[i]["STOM"] = np.log(result_list[i]["cshtrm"]/result_list[i]["cshom"])
    return result_list

In [26]:
def STOQ_func(unique_vals,result_list):
    for i in range(len(unique_vals)):
        result_list[i]["STOQ"] = np.NaN
        if len(result_list[i])>2:
            for j in reversed(range(len(result_list[i])-2)):
                x = np.exp(result_list[i]["STOM"].iloc[j+2])+np.exp(result_list[i]["STOM"].iloc[j+1])+np.exp(result_list[i]["STOM"].iloc[j])
                result_list[i].at[j,"STOQ"] = np.log(1/3*x)
    return result_list

# Management quality

### Asset growth

In [27]:
def AGRO_func(unique_vals,result_list):
    for i in range(len(unique_vals)):
        result_list[i]["AGRO"] = np.NaN
        if len(result_list[i])>19:
            result_list[i] = result_list[i].sort_index(ascending=False)
            #Make 5 years timeline for regression
            x = np.array([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]).reshape((-1, 1))
            temp_reg = np.zeros((len(result_list[i])))
            temp_sum = np.zeros((len(result_list[i])))
            k=0
            for j in range(len(result_list[i])-19):
                try:
                    temp = result_list[i]["atq"][k:k+20].to_numpy().reshape((-1, 1))
                    model = LinearRegression().fit(x, temp)
                    temp_reg[j] = model.coef_*(-1)                
                    temp_sum[j] = (result_list[i]["atq"][k:k+20].sum())/20
                    k = k+1

                except:
                    temp_sum[j] = np.NaN
                    temp_reg[j] = np.NaN
                    k = k+1

            result_list[i]["AGRO"] = temp_reg/temp_sum
            result_list[i] = result_list[i].sort_index(ascending=True)
    return result_list

### Insuance growth

In [28]:
def IGRO_func(unique_vals,result_list):
    for i in range(len(unique_vals)):
        result_list[i]["IGRO"] = np.NaN
        if len(result_list[i])>19:
            result_list[i] = result_list[i].sort_index(ascending=False)
            #Make 5 years timeline for regression
            x = np.array([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]).reshape((-1, 1))
            temp_reg = np.zeros((len(result_list[i])))
            temp_sum = np.zeros((len(result_list[i])))
            k=0
            for j in range(len(result_list[i])-19):
                try:
                    temp = result_list[i]["cshoq"][k:k+20].to_numpy().reshape((-1, 1))
                    model = LinearRegression().fit(x, temp)
                    temp_reg[j] = model.coef_*(-1)                
                    temp_sum[j] = (result_list[i]["cshoq"][k:k+20].sum())/20
                    k = k+1

                except:
                    temp_sum[j] = np.NaN
                    temp_reg[j] = np.NaN
                    k = k+1

            result_list[i]["IGRO"] = temp_reg/temp_sum
            result_list[i] = result_list[i].sort_index(ascending=True)
    return result_list

### Capital expenditure growth

In [29]:
def CXGRO_func(unique_vals,result_list): 
    for i in range(len(unique_vals)):
        result_list[i]["CXGRO"] = np.NaN
        if len(result_list[i])>19:
            result_list[i] = result_list[i].sort_index(ascending=False)
            #Make 5 years timeline for regression
            x = np.array([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]).reshape((-1, 1))
            temp_reg = np.zeros((len(result_list[i])))
            temp_sum = np.zeros((len(result_list[i])))
            k=0
            for j in range(len(result_list[i])-19):
                try:
                    temp = result_list[i]["capxy"][k:k+20].to_numpy().reshape((-1, 1))
                    model = LinearRegression().fit(x, temp)
                    temp_reg[j] = model.coef_*(-1)                
                    temp_sum[j] = (result_list[i]["capxy"][k:k+20].sum())/20
                    k = k+1

                except:
                    temp_sum[j] = np.NaN
                    temp_reg[j] = np.NaN
                    k = k+1

            result_list[i]["CXGRO"] = temp_reg/temp_sum
            result_list[i] = result_list[i].sort_index(ascending=True)
    return result_list

### Capital expenditure

In [30]:
def CX_func(unique_vals,result_list):
    for i in range(len(unique_vals)):
        result_list[i] = result_list[i].sort_index(ascending=False)
        result_list[i]["CX"] = np.NaN
        if len(result_list[i])>19:
            for j in range(len(result_list[i])-19):
                result_list[i].at[j,"CX"] = result_list[i]["capxy"].iloc[j]/((result_list[i]["capxy"][j:j+20].sum())/20)
        
        result_list[i] = result_list[i].sort_index(ascending=True)
    return result_list

# Profitability

### Asset turnover

In [31]:
def ATO_func(unique_vals,result_list):
    for i in range(len(unique_vals)):
        result_list[i]["ATO"] = result_list[i]["saleq"] / result_list[i]["atq"]
    return result_list

### Gross profitability

In [32]:
def GP_func(unique_vals,result_list):
    for i in range(len(unique_vals)):
        result_list[i]["GP"] = (result_list[i]["saleq"]-result_list[i]["cogsq"]) / result_list[i]["atq"]
    return result_list

### Gross margin

In [33]:
def GM_func(unique_vals,result_list):
    for i in range(len(unique_vals)):
        result_list[i]["GM"] = (result_list[i]["saleq"]-result_list[i]["cogsq"]) / result_list[i]["saleq"]
    return result_list    

# Prospect

### Maximum drawdown (12- Months)

In [36]:
def MAD_func(unique_vals,result_list):
    for i in range(len(unique_vals)):
        result_list[i]["MAD"] = np.NaN
        if len(result_list[i])>11:
            for j in reversed(range(len(result_list[i])-11)):
                result_list[i].at[j+11,"MAD"] = result_list[i]["prccm"][j:j+12].max()-result_list[i]["prccm"][j:j+12].min()
    return result_list

# Seasonality

In [37]:
def SEASON_func(unique_vals,result_list):
    for i in range(len(unique_vals)):
        result_list[i]["Total average"] = np.NaN
        result_list[i]["Seasonal average"] = np.NaN
        result_list[i]["SEASON"] = np.NaN
        if len(result_list[i])>59:
            for j in reversed(range(len(result_list[i])-60)):
                result_list[i].at[j+60,"Total average"] = result_list[i]["trt1m"][j+1:j+61].sum()/60
                result_list[i].at[j+60,"Seasonal average"] = result_list[i]["trt1m"].iloc[j+12:j+60+12:12].sum()/5

            result_list[i]["SEASON"] = result_list[i]["Seasonal average"]/result_list[i]["Total average"]
            
    return result_list

# Size

### Log of market cap

In [38]:
def LNCAP_func(unique_vals,result_list):
    for i in range(len(unique_vals)):
        result_list[i]["LNCAP"] =  np.log(result_list[i]["prccm"] * result_list[i]["cshom"])
    return result_list   

# Value

### Book-to-price ratio

In [39]:
def BTOP_func(unique_vals,result_list):
    for i in range(len(unique_vals)):
        result_list[i]["BTOP"] = (result_list[i]["atq"]-result_list[i]["ltq"])/(result_list[i]["cshoq"]*result_list[i]["prccq"])
    return result_list     

### Sales-to-price ratio

In [40]:
def STOP_func(unique_vals,result_list):
    for i in range(len(unique_vals)):
        result_list[i]["STOP"] =  result_list[i]["saleq"]/(result_list[i]["cshoq"]*result_list[i]["prccq"])
    return result_list    

# Volatility & momentum

### Annualized volatility

In [48]:
def annualized_volatility_func(unique_vals,result_list):
    for i in range(len(unique_vals)):
        result_list[i]["ANN VOL"] = np.NaN
        if len(result_list[i])>11:
            for j in range(11,len(result_list[i])):
                result_list[i].at[j,"ANN VOL"] = np.std(result_list[i]["trt1m"][j-11:j+1])*np.sqrt(12)            
    return result_list

### 1-month & 1-year momentum

In [49]:
def momentum_func(unique_vals,result_list):
    for i in range(len(unique_vals)):
        result_list[i]["1M MOM"] = result_list[i]["prccm"]-result_list[i]["prccm"].shift(1)
        result_list[i]["1Y MOM"] = result_list[i]["prccm"]-result_list[i]["prccm"].shift(12)
    return result_list

### Short and long term reversal, medium momentum

In [50]:
def slm_func(unique_vals,result_list):
    for i in range(len(unique_vals)):
        result_list[i]["SHOREV"] = result_list[i]["trt1m"].shift(1)
        result_list[i]["MED MOM"] = result_list[i]["prccm"].shift(2)/result_list[i]["prccm"].shift(12)-1
        result_list[i]["LONREV"] = result_list[i]["prccm"].shift(13)/result_list[i]["prccm"].shift(60)-1
    return result_list

### Downside Risk

In [73]:
def downside_func(unique_vals,result_list):
    exp_ret = 0.06
    for i in range(len(unique_vals)):
        result_list[i]["deviation"] = result_list[i]["trt1m"]-exp_ret
        result_list[i]["DOWNSIDE"] = np.abs(result_list[i][result_list[i]["deviation"]<0]["deviation"])
    return result_list

# Manipulations

In [74]:
#Function for running all descriptors
def run_all(unique_vals,result_list):
    result_list = DTOP_func(unique_vals,result_list)
    result_list = ABS_func(unique_vals,result_list)
    result_list = VSAL_func(unique_vals,result_list)
    result_list = DTOA_func(unique_vals,result_list)
    result_list = EGRO_func(unique_vals,result_list)
    result_list = SGRO_func(unique_vals,result_list)
    result_list = STOM_func(unique_vals,result_list)
    result_list = STOQ_func(unique_vals,result_list)
    result_list = AGRO_func(unique_vals,result_list)
    result_list = IGRO_func(unique_vals,result_list)
    result_list = CXGRO_func(unique_vals,result_list)
    result_list = CX_func(unique_vals,result_list)
    result_list = ATO_func(unique_vals,result_list)
    result_list = GP_func(unique_vals,result_list)
    result_list = GM_func(unique_vals,result_list)
    result_list = MAD_func(unique_vals,result_list)
    result_list = SEASON_func(unique_vals,result_list)
    result_list = LNCAP_func(unique_vals,result_list)
    result_list = BTOP_func(unique_vals,result_list)
    result_list = STOP_func(unique_vals,result_list)
    result_list = annualized_volatility_func(unique_vals,result_list)
    result_list = momentum_func(unique_vals,result_list)
    result_list = slm_func(unique_vals,result_list)
    result_list = downside_func(unique_vals,result_list)
    result_list = insert_tic_and_fill(result_list,unique_vals)
    return result_list

#Insert ticker
def insert_tic_and_fill(result_list,unique_vals):
    for i in range(len(unique_vals)):
        result_list[i]["tic"] = unique_vals[i]
        result_list[i] = result_list[i].fillna(method='ffill')
    return result_list

# Quarterly + yearly data

In [41]:
quarterly_all = pd.read_csv("QUATERLY ALL RAW.csv"); annual_all = pd.read_csv("ANNUAL ALL RAW.csv")

Fed_funds_raw = pd.read_csv("FEDFUNDS.csv") ; Fed_funds_raw["datadate"] = pd.to_datetime(Fed_funds_raw["DATE"]) ; Fed_funds_raw = Fed_funds_raw.set_index("datadate")
GDP_raw = pd.read_csv("GDP.csv") ; GDP_raw["datadate"] = pd.to_datetime(GDP_raw["DATE"]) ; GDP_raw = GDP_raw.set_index("datadate")
TB3MS_raw = pd.read_csv("TB3MS.csv") ; TB3MS_raw["datadate"] = pd.to_datetime(TB3MS_raw["DATE"]) ; TB3MS_raw = TB3MS_raw.set_index("datadate")
DGS3_raw = pd.read_csv("DGS3.csv") ; DGS3_raw["datadate"] = pd.to_datetime(DGS3_raw["DATE"]) ; DGS3_raw = DGS3_raw.set_index("datadate")
DGS10_raw = pd.read_csv("DGS10.csv") ; DGS10_raw["datadate"] = pd.to_datetime(DGS10_raw["DATE"]) ; DGS10_raw = DGS10_raw.set_index("datadate")
UNRATE_raw = pd.read_csv("UNRATE.csv"); UNRATE_raw["datadate"] = pd.to_datetime(UNRATE_raw["DATE"]) ; UNRATE_raw = UNRATE_raw.set_index("datadate")

Macro_df = pd.concat([Fed_funds_raw,GDP_raw["GDP_PCH"],TB3MS_raw["TB3MS"],DGS3_raw["DGS3"],DGS10_raw["DGS10"],UNRATE_raw["UNRATE"]],axis=1)
Macro_df = Macro_df[:-1] #Delete last row since this is empty
Macro_df['DGS3'] = Macro_df['DGS3'].astype(float) #convert string to float
Macro_df["DGS_diff"] = Macro_df["DGS3"]-Macro_df["DGS10"]
Macro_df["rf"] = Macro_df["DGS10"]
Macro_df = Macro_df.drop(["DGS3","DGS10"],axis=1)
Macro_df = Macro_df.shift(-1)


quarterly_filtered = quarterly_all.loc[((quarterly_all['exchg'] == 11.0) | (quarterly_all['exchg'] == 12.0) | (quarterly_all['exchg'] == 14.0) )]
annual_filtered = annual_all.loc[((annual_all['exchg'] == 11.0) | (annual_all['exchg'] == 12.0) | (annual_all['exchg'] == 14.0) )]

quarterly_filtered = quarterly_filtered[~pd.isnull(quarterly_filtered["tic"])]; annual_filtered = annual_filtered[~pd.isnull(annual_filtered["tic"])]
quarterly_filtered = quarterly_filtered.drop_duplicates(); annual_filtered = annual_filtered.drop_duplicates()

quarterly_filtered = quarterly_filtered.sort_values(["tic","datadate"]); annual_filtered = annual_filtered.sort_values(["tic","datadate"])

quarterly_filtered["datadate"] = pd.to_datetime(quarterly_filtered["datadate"]) ; annual_filtered["datadate"] = pd.to_datetime(annual_filtered["datadate"])

quarterly_filtered = quarterly_filtered.reset_index(drop = True) ; annual_filtered = annual_filtered.reset_index(drop = True)
quarterly_filtered = quarterly_filtered.set_index("datadate") ; annual_filtered = annual_filtered.set_index("datadate")

# Monthly data

In [42]:
monthly_all = pd.read_csv("MONTHLY ALL RAW.csv")
monthly_all["trt1m"] = monthly_all["trt1m"]/100
monthly_all = monthly_all.drop_duplicates()
monthly_all = monthly_all.reset_index(drop = True)
monthly_all["datadate"] = pd.to_datetime(monthly_all["datadate"])
monthly_all = monthly_all.set_index("datadate")

monthly_all_merged = pd.merge_asof(monthly_all.sort_index(), Macro_df, on="datadate")
monthly_all_merged["tic"] = monthly_all_merged["tic"].astype(str) 
monthly_all_merged['FEDFUNDS'].mask(monthly_all_merged['datadate'] > '2023-02-01', np.nan, inplace=True)
monthly_all_merged['GDP_PCH'].mask(monthly_all_merged['datadate'] > '2023-02-01', np.nan, inplace=True)
monthly_all_merged['TB3MS'].mask(monthly_all_merged['datadate'] > '2023-02-01', np.nan, inplace=True)
monthly_all_merged['DGS_diff'].mask(monthly_all_merged['datadate'] > '2023-02-01', np.nan, inplace=True)
monthly_all_merged['UNRATE'].mask(monthly_all_merged['datadate'] > '2023-02-01', np.nan, inplace=True)
monthly_all_merged['rf'].mask(monthly_all_merged['datadate'] > '2023-02-01', np.nan, inplace=True)
monthly_all_merged = monthly_all_merged.set_index("datadate")

# Execute

In [43]:
unique_vals_mon = unique_vals_func(monthly_all_merged)
unique_vals_quarterly = unique_vals_func(quarterly_filtered)
unique_vals_quarterly = unique_vals_quarterly[~pd.isnull(unique_vals_quarterly)]
unique_vals = np.intersect1d(unique_vals_mon,unique_vals_quarterly)
monthly_all_merged = monthly_all_merged[monthly_all_merged["tic"].isin(unique_vals)]

result_list = dic_func(unique_vals)
result_list = insert_data(quarterly_filtered,monthly_all_merged,unique_vals,result_list)


# remove inconsistensies
for i in range(len(unique_vals)):
    result_list[i][["atq","prccq","epspxq","saleq","cshoq","cshom","prccm"]] = result_list[i][["atq","prccq","epspxq","saleq","cshoq","cshom","prccm"]].replace({0: np.nan})
    result_list[i][["atq","prccq","epspxq","saleq","cshoq","rf","cshom","prccm"]] = result_list[i][["atq","prccq","epspxq","saleq","cshoq","rf","cshom","prccm"]].fillna(method='ffill')
    result_list[i][["atq","prccq","epspxq","saleq","cshoq","rf","cshom","prccm"]] = result_list[i][["atq","prccq","epspxq","saleq","cshoq","rf","cshom","prccm"]].fillna(method='bfill')

  0%|          | 0/6417 [00:00<?, ?it/s]

In [75]:
result_list = run_all(unique_vals,result_list)

0/23
1/23
2/23
3/23
4/23
5/23
6/23
7/23
8/23
9/23
10/23
11/23
12/23
13/23
14/23
15/23
16/23
17/23
18/23
19/23
20/23
21/23
22/23
23/23
24/23
25/23
26/23


# Create final data

In [78]:
result_df_raw = pd.concat(result_list)
#Make datadate the index
#result_df_raw["datadate"] = pd.to_datetime(result_df_raw["index"])
#result_df_raw = result_df_raw.drop(["index"],axis=1)
result_df_raw = result_df_raw.set_index("datadate")
result_df_raw = result_df_raw.sort_index()

result_df = result_df_raw[["tic","MIDREV","MIDREV excess","DTOP","ABS","VSAL","DTOA","EGRO","SGRO","STOM","STOQ","AGRO","IGRO","CXGRO","CX","ATO","GP","GM","MAD","SEASON","LNCAP","BTOP","STOP","ANN VOL","1M MOM","1Y MOM","SHOREV","MED MOM","LONREV","DOWNSIDE","return quarterly","trt1m","FEDFUNDS","GDP_PCH","TB3MS","DGS_diff","UNRATE","rf","ggroup","gsector","naics","sic"]]
#result_df_GIC = result_df_raw[["tic","ggroup","gind","gsector","gsubind"]]

#Output final csv file
result_df.to_csv("sorted_dates.csv")
#result_df_raw.to_csv("results_raw.csv")

In [ ]:
#If one wants to load in the data in this jupytor notebook
df = pd.read_csv("sorted_dates.csv") # Load in unfinished dataset

df_vix = pd.read_csv("VIXCLS.csv") # Load in VIX (we decided to add it late)
# Make manipulations to VIX df and merge it to dataset
df_vix["datadate"] = df_vix["DATE"]
df_vix["datadate"] = pd.to_datetime(df_vix["datadate"])
df["datadate"] = pd.to_datetime(df["datadate"])
df_vix = df_vix.drop(["DATE"],axis = 1)
df = pd.merge_asof(df,df_vix, on ="datadate")

df_close = pd.read_csv("Closing_price_month.csv") # Load in closing price monthly (we noticed we might need it)
df_close["datadate"] = pd.to_datetime(df_close["datadate"])
df_close = df_close[["tic","datadate","prccm"]]

df = df.reset_index(drop=True)

df = df.fillna(0) #Fill NaN with 0
# Remove a bunch of unused columns from dataset
df = df.loc[:, ~df.columns.isin(["trt1m","return quarterly","tic.1","ggroup.1", 'ggroup', "gsector","gsector.1","naics.1","naics","sic.1","sic"])]
df = df.sort_values(["tic","datadate"]) # Sort dataset on tickers and datadate, to get in correct order
df = pd.merge(df, df_close, on=['tic', 'datadate']) # Merge closing price to dataset
# Remove two firms with wildly inconsistent price and return data
df = df[df["tic"]!="CRGE"]
df = df[df["tic"]!="HYMC"]

df["log_ret"] = np.log(df["prccm"]/df["prccm"].shift(1)) # Include log-return from "current" month as feature
df["target"] = np.log(df["prccm"].shift(-1)/df["prccm"]) # We want to "forecast" (predict next month's log-return)

# Load in monthly dataset since we need some information from here for the portfolio stuff
monthly = pd.read_csv("MONTHLY ALL RAW.csv")
monthly = monthly[["tic","datadate","prccm","cshom"]] # Only interested in ticker, datadate, closing price, and shares outstanding
monthly["mkt cap"] = monthly["prccm"] * monthly["cshom"] # Calculate market cap (used in market cap weighted portfolios)
monthly = monthly.drop(["prccm","cshom"],axis=1) # After market cap is calculated, we have no need for closing price and shares outstanding
# Make manipulations and merge to dataset
monthly["datadate"] = pd.to_datetime(monthly["datadate"]) 
df = pd.merge(df, monthly, on=['tic', 'datadate'])
df["mkt cap"] = df["mkt cap"].fillna(0) # We noticed a few places with NaN values in market cap, remove these


# The next few lines correct an issue we had, where, when calculating log-returns, some firms' final datapoints would use
# the incorrect data from another firm. This is obviously not correct, so we remove every firm's final datapoint
df = df.reset_index(drop = True)
temp = [0] # Create temp for storing index
for i in tqdm_notebook(range(len(df)-1)):
    if df["tic"].iloc[i] != df["tic"].iloc[i+1]: # Find where we need to delete a datapoint (where NaN should have appead)
        temp.append(i) # Append the index
        temp.append(i+1)
temp.append(len(df)-1)
df = df.drop(temp) # Remove the index from df
df = df.dropna() # Remove NaN values
df = df.reset_index(drop = True)
df = df.sort_values(["tic","datadate"]) # Sort dataset on tickers and datadate, to get in correct order, again

n_features = 31 # Choose amount of features for use in later function. We have 31

input_size = 12 # Determine the amount of data for each firm that should på put in the model. We choose 12 months
train_size = 0.8 # NOT USED!!! - old variable from when we trained on 80% and tested on 20% of data

df = df[df["datadate"]<"2023"] # We decided for simplicity to cut off the data at 2023

# The next lines are what separates the data in the aforementioned periods
#df = df[(df["datadate"]<"2012-01-01") & (df["datadate"]>"2004-02-01")] 
#df = df[(df["datadate"]<"2019-01-01") & (df["datadate"]>"2011-02-01")] 
#df = df[(df["datadate"]<"2023-01-01") & (df["datadate"]>"2018-02-01")] 

df = df[(df["datadate"]<"2005") & (df["datadate"]>"1989")] 

# After examining descriptor correlation, these were deemed not important, and thus dropped
df = df.drop(["STOQ","ATO","TB3MS","rf"],axis = 1)

# Correlation plot

In [ ]:
df = pd.read_csv("sorted_dates.csv")
df_vix = pd.read_csv("VIXCLS.csv")
df_vix["datadate"] = df_vix["DATE"]
df_vix["datadate"] = pd.to_datetime(df_vix["datadate"])
df["datadate"] = pd.to_datetime(df["datadate"])
df_vix = df_vix.drop(["DATE"],axis = 1)
df = pd.merge_asof(df,df_vix, on ="datadate")
df = df[df["trt1m"].notna()]
df = df[df["trt1m"]<20]
df = df.reset_index(drop=True)
df = df.fillna(0)
df = df.loc[:, ~df.columns.isin(["return quarterly","tic.1","ggroup.1", 'ggroup', "gsector","gsector.1","naics.1","naics","sic.1","sic"])]
df1 = df.pop('trt1m') # remove column trt1m and store it in df1
df['trt1m']=df1 # add trt1m series as a 'new' column.
df = pd.read_csv("Final_data.csv")
df = df.drop(["Unnamed: 0","rf","DOWNSIDE","target"],axis = 1)
corr_matrix = df.corr()
plt.figure(figsize=(40,40))
sn.heatmap(corr_matrix, annot=True)
np.tril(np.ones(corr_matrix.shape)).astype(bool)
df_lt = corr_matrix.where(np.tril(np.ones(corr_matrix.shape)).astype(bool))
plt.figure(1,figsize=(20,20))
ax=sn.heatmap(df_lt,annot=True,fmt='.1g',cmap="coolwarm",square=True,cbar_kws={"shrink": 0.825})
for text in ax.texts:
    text.set_size(6.5)

# Unique firms plot

In [ ]:
years = np.arange(1990,2023,1)
distribution_tics = []
for year in years:
    x = len(np.unique(df[(df["datadate"]>str(year)) & (df["datadate"]<str(year+1))]["tic"]))
    distribution_tics.append(x)
plt.figure(figsize=(20,10))
plt.bar(years,distribution_tics)
plt.axhline(y = np.mean(distribution_tics), color = 'k', linestyle = '--',label="Mean no. of firms per year")
plt.xlim(1989,2023)
years_3 = np.arange(1990,2023,3)
plt.xticks(ticks=years_3, labels=years_3)
plt.title("Number of unique firms each year",fontsize=28)
plt.grid(axis="y",alpha=0.5)
plt.rc('xtick', labelsize=20) 
plt.rc('ytick', labelsize=20)
plt.legend(loc=2)


In [ ]:
data = pd.read_csv("testtestset.csv")
data = data.loc[((data['exchg'] == 11.0) | (data['exchg'] == 12.0) | (data['exchg'] == 14.0) )]
data = data[~pd.isnull(data["tic"])]
data["datadate"] = pd.to_datetime(data["datadate"])
years = np.arange(1990,2023,1)
distribution_tics_inac = []
for year in years:
    x = len(np.unique(data[(data["datadate"]>str(year)) & (data["datadate"]<str(year+1))]["tic"]))
    distribution_tics_inac.append(x)
    print(str(year)+":",x,"unique tickers")
print("Total unique tickers:",len(np.unique(data["tic"])))

In [ ]:
plt.figure(figsize=(20,10))
plt.bar(years,distribution_tics_inac)
plt.axhline(y = np.mean(distribution_tics_inac), color = 'k', linestyle = '--',label="Mean no. of firms per year")
plt.xlim(1989,2024)
years_3 = np.arange(1990,2023,3)
plt.xticks(ticks=years_3, labels=years_3)
plt.title("Number of unique firms each year",fontsize=28)
plt.grid(axis="y",alpha=0.5)
plt.rc('xtick', labelsize=20) 
plt.rc('ytick', labelsize=20)
plt.legend(loc=2)

In [ ]:
plt.figure(figsize=(30,15))
# Plot bar graphs of unique firm distributions
plt.bar(years,distribution_tics_inac,color="cornflowerblue",edgecolor="k",hatch="/",label="# of firms each year including inactive firms")
plt.bar(years,distribution_tics,color="indianred",edgecolor="k", label="# of firms each year with inactive firms dropped")
# Plot horisontal lines of mean # of unique firms
plt.axhline(y = np.mean(distribution_tics_inac), linestyle = '--',color="k",label="Mean # of firms per year, including inactive firms ("+str(int(np.mean(distribution_tics_inac)))+")")
plt.axhline(y = np.mean(distribution_tics), linestyle = '-',color="k",label="Mean # of firms per year, with inactive firms dropped ("+str(int(np.mean(distribution_tics)))+")")
# Manipulate plot axis limits
plt.xlim(1989,2023)
plt.ylim(0,10000)
# Define units for both x- and y axis
years_3 = np.arange(1990,2023,2)
plt.xticks(ticks=years_3, labels=years_3)
y_ticks = [0,1000,2000,3000,4000,5000,6000,7000,8000]
plt.yticks(ticks=y_ticks, labels=y_ticks)
# Give plot a grid, and axis font-sizes
plt.grid(axis="y",alpha=0.5)
plt.rc('xtick', labelsize=20) 
plt.rc('ytick', labelsize=25)
# The following bit of code changes the order that things are shown in the legend
#----------------------------------
#get handles and labels
handles, labels = plt.gca().get_legend_handles_labels()
#specify order of items in legend
order = [2,3,0,1]
#add legend to plot
plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order],loc=0,fontsize=30) 
#----------------------------------

In [ ]:
#create test train set
from functools import reduce
tics = np.unique(df["tic"])
N100 = pd.ExcelFile("Ticker NASDAQ100 data BLOOMBERG.xlsx")
Nasdaq100 = {}
years = np.arange(2005,2023)
Tickers = []
nasdaq_total = []
nasdaq_wehave = []
for year in years:
    Nasdaq100[year] = pd.read_excel(N100, str(year))
    for j in range(len(Nasdaq100[year])):
        Nasdaq100[year].at[j,"Ticker"] = Nasdaq100[year]["Ticker"].tolist()[j].split()[0]
    nasdaq_total.append(len(Nasdaq100[year]["Ticker"]))
    nasdaq_wehave.append(len(np.intersect1d(np.unique(df["tic"]),Nasdaq100[year]["Ticker"])))


In [ ]:
plt.figure(figsize=(30,15))
# Plot bar graphs of unique firm distributions
plt.bar(years,nasdaq_total,color="cornflowerblue",edgecolor="k",hatch="/",label="# of firms each year in Nasdaq 100")
plt.bar(years,nasdaq_wehave,color="indianred",edgecolor="k", label="# of firms each year from Nasdaq 100, that we have in dataset")
# Plot horisontal lines of mean # of unique firms
plt.axhline(y = np.mean(nasdaq_total), linestyle = '--',color="k",label="Mean # of firms per year in Nasdaq 100 ("+str(int(np.mean(nasdaq_total)))+")")
plt.axhline(y = np.mean(nasdaq_wehave), linestyle = '-',color="k",label="Mean # of firms per year from Nasdaq 100, that we have in dataset ("+str(int(np.mean(nasdaq_wehave)))+")")
# Manipulate plot axis limits
plt.xlim(2004,2023)
plt.ylim(0,150)
# Define units for both x- and y axis
years_3 = np.arange(2005,2023,1)
plt.xticks(ticks=years_3, labels=years_3)
y_ticks = [0,20,40,60,80,100]
plt.yticks(ticks=y_ticks, labels=y_ticks)
# Give plot a grid, and axis font-sizes
plt.grid(axis="y",alpha=0.5)
plt.rc('xtick', labelsize=25) 
plt.rc('ytick', labelsize=25)
# The following bit of code changes the order that things are shown in the legend
#----------------------------------
#get handles and labels
handles, labels = plt.gca().get_legend_handles_labels()
#specify order of items in legend
order = [2,3,0,1]
#add legend to plot
plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order],loc=2,fontsize=30) 
#----------------------------------

# Macro features plot

In [ ]:
Fed_funds_raw = pd.read_csv("FEDFUNDS.csv") ; Fed_funds_raw["datadate"] = pd.to_datetime(Fed_funds_raw["DATE"]) ; Fed_funds_raw = Fed_funds_raw.set_index("datadate")
GDP_raw = pd.read_csv("GDP.csv") ; GDP_raw["datadate"] = pd.to_datetime(GDP_raw["DATE"]) ; GDP_raw = GDP_raw.set_index("datadate")
TB3MS_raw = pd.read_csv("TB3MS.csv") ; TB3MS_raw["datadate"] = pd.to_datetime(TB3MS_raw["DATE"]) ; TB3MS_raw = TB3MS_raw.set_index("datadate")
DGS3_raw = pd.read_csv("DGS3.csv") ; DGS3_raw["datadate"] = pd.to_datetime(DGS3_raw["DATE"]) ; DGS3_raw = DGS3_raw.set_index("datadate")
DGS10_raw = pd.read_csv("DGS10.csv") ; DGS10_raw["datadate"] = pd.to_datetime(DGS10_raw["DATE"]) ; DGS10_raw = DGS10_raw.set_index("datadate")
UNRATE_raw = pd.read_csv("UNRATE.csv"); UNRATE_raw["datadate"] = pd.to_datetime(UNRATE_raw["DATE"]) ; UNRATE_raw = UNRATE_raw.set_index("datadate")

Macro_df = pd.concat([Fed_funds_raw,GDP_raw["GDP_PCH"],TB3MS_raw["TB3MS"],DGS3_raw["DGS3"],DGS10_raw["DGS10"],UNRATE_raw["UNRATE"]],axis=1)
Macro_df = Macro_df[:-1] #Delete last row since this is empty
Macro_df['DGS3'] = Macro_df['DGS3'].astype(float) #convert string to float
Macro_df["DGS_diff"] = Macro_df["DGS3"]-Macro_df["DGS10"]
Macro_df["rf"] = Macro_df["DGS10"]
Macro_df = Macro_df.drop(["DGS3","DGS10"],axis=1)
Macro_df = Macro_df.shift(-1)
#Macro_df = Macro_df[:-1] #Delete last row since this is empty
#Macro_df = Macro_df.fillna(method="ffill")

In [ ]:
from matplotlib.patches import ConnectionPatch

In [ ]:
fig = plt.figure(figsize=(20,20))
fig.patch.set_facecolor('white')
ax1 = plt.subplot(2,1,1)
plt.plot(Macro_df["rf"],label="rf")
plt.plot(Macro_df["DGS_diff"],label="DGS_diff",linewidth=2)
plt.plot(Macro_df["FEDFUNDS"],label="FEDFUNDS",linewidth=2)
plt.plot(Macro_df["TB3MS"],label="TB3MS",linewidth=2)
plt.plot(Macro_df["GDP_PCH"].dropna(),label="GDP_PCH",linewidth=2)
plt.plot(Macro_df["UNRATE"],label="UNRATE",linewidth=2)
# Plot horisontal line on y=0
plt.axhline(y = 0, color = 'k', linestyle = '--')
# Define intervals for US recessions (from https://fredhelp.stlouisfed.org/fred/data/understanding-the-data/recession-bars/)
# index is in days starting with 0 from 1970-01-01, no. of days dalculated from https://planetcalc.com/9788/
rec_1 = [7485,7728] #rec_1: 1990-07-01, 1991-03-01
rec_2 = [11381,11626] #rec_2: 2001-03-01, 2001-11-01
rec_3 = [13847,14395] #rec_3: 2007-12-01, 2009-06-01
rec_4 = [18292,18352] #rec_4: 2020-02-01, 2020-04-01
plt.fill_between(rec_1, 0.17, -0.11, facecolor=(0,0,0,.2), edgecolor=(0,0,0),zorder=2) 
plt.fill_between(rec_2, 0.17, -0.11, facecolor=(0,0,0,.2), edgecolor=(0,0,0),zorder=2)
plt.fill_between(rec_3, 0.17, -0.11, facecolor=(0,0,0,.2), edgecolor=(0,0,0),zorder=2)
plt.fill_between(rec_4, 0.17, -0.11, facecolor=(0,0,0,.2), edgecolor=(0,0,0),zorder=2)

plt.xlabel("Year",fontsize=25)
# Change what is shown on x-axis
x_ticks = [7304,8400,9495,10591,11687,12783,13878,14974,16070,17166,18261,19357]
years = np.arange(1990,2024,3)
plt.xticks(ticks=x_ticks, labels=years)
plt.rc('xtick', labelsize=15) 
plt.rc('ytick', labelsize=15)
plt.grid(zorder=3)
plt.ylim(-0.1,0.16)
plt.xlim(7304,19388)
plt.legend(loc=(0.05,0.7),fontsize=15)


ax2 = plt.subplot(2,1,2)
plt.plot(Macro_df["rf"],label="rf")
plt.plot(Macro_df["DGS_diff"],label="DGS_diff",linewidth=2)
plt.plot(Macro_df["FEDFUNDS"],label="FEDFUNDS",linewidth=2)
plt.plot(Macro_df["TB3MS"],label="TB3MS",linewidth=2)
plt.plot(Macro_df["GDP_PCH"].dropna(),label="GDP_PCH",linewidth=2)
plt.plot(Macro_df["UNRATE"],label="UNRATE",linewidth=2)
plt.axhline(y = 0, color = 'k', linestyle = '--')
# Define intervals for US recessions (from https://fredhelp.stlouisfed.org/fred/data/understanding-the-data/recession-bars/)
#rec_4: 2020-02-01, 2020-04-01
# index is in days starting with 0 from 1970-01-01, # days dalculated from https://planetcalc.com/9788/
rec_4 = [18292,18352]
plt.fill_between(rec_4, 0.17, -0.11, facecolor=(0,0,0,.3), edgecolor=(0,0,0),zorder=2)

#plt.title("Macroeconomic Features",fontsize=36)
plt.xlabel("Date",fontsize=25)
plt.grid(zorder=3)
x_ticks = [18139,18200,18261,18321,18382,18443,18505]
years = ["2019-09","2019-11","2020-01","2020-03","2020-05","2020-07","2020-09"]
plt.xticks(ticks=x_ticks, labels=years)
plt.rc('xtick', labelsize=15) 
plt.rc('ytick', labelsize=15)
plt.ylim(-0.1,0.16)
plt.xlim(18139,18505)
plt.legend(loc=(0.05,0.6),fontsize=20)

# Add line from one subplot to the other
xyA = [18139, -0.101]
xyB = [18139, 0.16]
# ConnectionPatch handles the transform internally so no need to get fig.transFigure
line = ConnectionPatch(
    xyA,
    xyB,
    coordsA=ax1.transData,
    coordsB=ax2.transData,
    # Default shrink parameter is 0 so can be omitted
    color=(0,0,0,1),
    linestyle="--",  # "normal" arrow
    mutation_scale=10,  # controls arrow head size
    linewidth=2,
)
fig.patches.append(line)

# Add line from one subplot to the other
xyA = [18505, -0.101]
xyB = [18505, 0.16]
# ConnectionPatch handles the transform internally so no need to get fig.transFigure
line = ConnectionPatch(
    xyA,
    xyB,
    coordsA=ax1.transData,
    coordsB=ax2.transData,
    # Default shrink parameter is 0 so can be omitted
    color=(0,0,0,1),
    linestyle="--", # -|> "normal" arrow
    mutation_scale=10,  # controls arrow head size
    linewidth=2,
)
fig.patches.append(line)

# REST OF MODEL

Since we decided that we wanted to compare our portfolio results to the Nasdaq 100 index, it makes sense that we build our portfolio on the very same Nasdaq 100 constituents. Thus, we create a test set only containing the firms available in our dataset that were/are Nasdaq constituents in the correct time periods.

In [ ]:
#create test train set
from functools import reduce
N100 = pd.ExcelFile("Ticker NASDAQ100 data BLOOMBERG.xlsx") # Load in Nasdaq 100 constituents for every year between 2005 and 2023
Nasdaq100 = {} # Create dictionary that will contain the constituents for each year
# Divide out information from each excel sheet in dictionary 
years = np.arange(2004,2024)
Tickers = []
for year in years:
    Nasdaq100[year] = pd.read_excel(N100, str(year))
    for j in range(len(Nasdaq100[year])):
        Nasdaq100[year].at[j,"Ticker"] = Nasdaq100[year]["Ticker"].tolist()[j].split()[0]
    Tickers = reduce(np.union1d, (Tickers,Nasdaq100[year]["Ticker"])) # End up with complete list of Nasdaq 100 constituents
    
        
unique_tics = np.unique(df["tic"]) # Determine the unique tickers in dataset
intersect = np.intersect1d(unique_tics,Tickers) # Find the intersection between tickers in dataset and Nasdaq 100 constituents 
train = df[~df["tic"].isin(intersect)] # Create train set with every ticker NOT in Nasdaq 100
test = df[df["tic"].isin(intersect)] # Create test set with every ticker in Nasdaq 100
test_pf = df[df["tic"].isin(intersect)] # Create duplicate test set with every ticker in Nasdaq 100 for portfolio stuff
train = train.drop(["prccm","mkt cap"],axis = 1) # Remove close price and market cap, since these are not used for training 
test = test.drop(["prccm","mkt cap"],axis = 1) # Remove close price and market cap, since these are not used for testing
test_pf = test_pf[["datadate","tic","prccm","mkt cap"]] # Only keep datadate, ticker, close price, and market cap in duplicate test set used for portfolio stuff 

In [ ]:
# Scale both train and test set
for col in train.columns[2:-1]:
    scaler = MinMaxScaler()
    train[[col]] = scaler.fit_transform(train[[col]]) # Use fit_transform on training set
    test[[col]] = scaler.transform(test[[col]]) # Use transform on test set. ONLY NASDAQ COMPANIES

The next functions are what uses the input_size parameter from earlier. Basically, we want to represent the data in "chunks" of _input_size_ (this case 12) datapoints for each firm at a time. Thus, the first function creates a list, _dates_df_, of dataframes for the train set each containing a years worth of data with the next index of the list sliding one month ahead. The final output is as said a list where the index is the corresponding month's index (dates_df[0] contains the first month and so on)

The bottom function does roughly the same but for the test set. A key difference is, that the resulting list is a level deeper since we need to keep track of specific tickers for each year. Thus the first index corresponds to the year and the next index is the month (dates_df_test[0][0] will be first month of first year, while dates_df_test[1][11] is the last month of the next year and so on)

In [ ]:
def input_size_func(df):
    dates_df = [] # Allocate storage to save data for each month
    unique_dates = np.unique(df["datadate"]) # Determine the unique dates 
    for i in range(input_size-1,len(unique_dates)): # Iterate over every month but start at "input_size"-1 since we cant look 12 months back starting at month 0
        try:
            # Look at data in range of "input_size" sliding one month ahead at a time
            dates_df.append(df[(df["datadate"]>=unique_dates[i-input_size+1]) & (df["datadate"]<=unique_dates[i])].reset_index(drop=True))
        except:
            None
    return dates_df

def input_size_func_test(dictionary,test_set):
    input_size_test = [] # Allocate storage to save data for each month
    years = np.arange(2019,2023) #'''REMEMBER TO CHANGE!!!''' # Decide which years to iterate over (three respective periods)
    for year in years:
        temp=test_set[test_set["tic"].isin(np.unique(dictionary[year]["Ticker"]))] # Look at specific tickers every year        
        temp_test=temp[(temp["datadate"]>=str(year-1)+"-02-01") & (temp["datadate"]<str(year+1) )] # Look at data in range of "input_size" sliding one month ahead at a time
        input_size_test.append(input_size_func(temp_test)) # Use above function for each year, creating a list that will be 1 level deeper
    return input_size_test
        
dates_df_train = input_size_func(train)    
dates_df_test_dict = input_size_func_test(Nasdaq100,test)
dates_df_test_pf = input_size_func_test(Nasdaq100,test_pf)

We noticed a little too late on, that the way we created the _input_size_ data in the above functions, we did not make sure that we only had data in chunks of 12. Thus, some places would contain a chunk of some random number of data points. This created a bunch of issues, since we need chunks of 12. A quick fix was to create two functions that could remove the inconsistensies. 

In [ ]:
def remove_input_size_errors_train(dates_df_train):
    for l in tqdm_notebook(range(len(dates_df_train))): # Iterate over every index of list 
        k=0 # Assign start-index variable, so we can start next loop from index where last item is removed
        stop = 1 # Assign dummy variable to determine if we have removed something and can stop the current list index 
        # (stop == 0 means that we have not stopped in an entire run-through, and can then jump to next index) 
        for j in range(99999999):
            if stop == 0: # If we have not stopped in the previous run-through of current index there are no inconsitensies, we can jump to next index
                break
            stop = 0 # Set stop variable
            for i in range(k,len(dates_df_train[l]),input_size): # Jump 12 each step, start at k so we dont have to start from beginning when removing 
                try:
                    if dates_df_train[l].iloc[i]["tic"]!=dates_df_train[l].iloc[i+input_size-1]["tic"]: # If the firm 11 places in front is different we dont have chunk of 12
                        dates_df_train[l] = dates_df_train[l].drop(i).reset_index(drop=True) # Remove current index and reset index
                        k = i # Set start-index, since the next inconsistency will always come after the one we just removed
                        stop = 1 # Since we have removed datapoint, we have stopped, and thus it is not time to break out of loop since there might be more inconsitencies
                        break
                except:
                    dates_df_train[l] = dates_df_train[l].drop(i).reset_index(drop=True) # We will end up in except statement near the end if there is inconsistency. Remove this
                    k = i # Set start-index, since the next inconsistency will always come after the one we just removed
                    stop = 1 # Since we have removed datapoint, we have stopped, and thus it is not time to break out of loop since there might be more inconsitencies
    return dates_df_train

def remove_input_size_errors_test(dates_df_test):
    # Remember that this list is a level deeper, so we need to keep track of both year and month
    year = 0
    month = 0
    for l in tqdm_notebook(range(len(dates_df_test)*12)): # The total number of indexes is the amount of years multiplied with 12
        k=0 # Assign start-index variable, so we can start next loop from index where last item is removed
        stop = 1 # Assign dummy variable to determine if we have removed something and can stop the current list index
        for j in range(9999999):
            if stop == 0: # If we have not stopped in the previous run-through of current index there are no inconsitensies, we can jump to next index
                break
            stop = 0 # Set stop variable
            for i in range(k,len(dates_df_test[year][month]),input_size): # Jump 12 each step, start at k so we dont have to start from beginning when removing 
                try:
                    if dates_df_test[year][month].iloc[i]["tic"]!=dates_df_test[year][month].iloc[i+input_size-1]["tic"]: # If the firm 11 places in front is different we dont have chunk of 12
                        dates_df_test[year][month] = dates_df_test[year][month].drop(i).reset_index(drop=True) # Remove current index and reset index
                        k = i # Set start-index, since the next inconsistency will always come after the one we just removed
                        stop = 1 # Since we have removed datapoint, we have stopped, and thus it is not time to break out of loop since there might be more inconsitencies
                        break
                except:
                    dates_df_test[year][month] = dates_df_test[year][month].drop(i).reset_index(drop=True) # We will end up in except statement near the end if there is inconsistency. Remove this
                    k = i # Set start-index, since the next inconsistency will always come after the one we just removed
                    stop = 1 # Since we have removed datapoint, we have stopped, and thus it is not time to break out of loop since there might be more inconsitencies
        month = (month+1) % 12 # Update month (every time we reach 12 it will reset to 0 and begin a new year)
        year = int(np.floor((l+1)/12)) # Update year, only change when we reach 12th, 24th, ... index
    return dates_df_test

dates_df_train = remove_input_size_errors_train(dates_df_train)
dates_df_test_dict = remove_input_size_errors_test(dates_df_test_dict)
dates_df_test_pf = remove_input_size_errors_test(dates_df_test_pf)

The next functions split datasets into features and target, but also keeps individual firms separated. We want the data in a way where 12 months of features correspond to the last month's target. Example: We save features for one firm from Jan to Dec, and save the target for Dec.

Again, bottom function is a level deeper.

In [ ]:
def split(dictionary):
    # Create dictionaries that will contain features and target 
    X = {}
    y = {}
    for i in tqdm_notebook(range(len(dictionary))):
        # Create temporary lists that will contain dataframes containg chunks of 12 for each individual firm, so 1 dataframe per firm
        save_X = []
        save_y = []
        try:
            for j in range(len(dictionary[i])):
                # If next firm is different, we need to save the features from previous 11 months and from current month + target from current month
                if dictionary[i]["tic"].iloc[j] != dictionary[i]["tic"].iloc[j+1]: 
                    save_X.append(dictionary[i].iloc[j-input_size+1:j+1,2:-1]) # Save features 
                    save_y.append(dictionary[i].iloc[j,-1]) # Save target
                    
        except: # We end up in the except statement in the end
            save_X.append(dictionary[i].iloc[j-input_size+1:j+1,2:-1]) # Save features 
            save_y.append(dictionary[i].iloc[j,-1]) # Save target
        #Save temporary lists in dictionary
        X[i] = save_X 
        y[i] = save_y
    return X,y

def split_test(test_dict):
    # Create dictionaries that will contain features and target 
    X_temp,y_temp = {},{}
    for k in range(len(test_dict)):
        X_,y_ = split(test_dict[k]) # Use above function for every year of test set
        X_temp[k] = X_ # Save features for each year in dictionary
        y_temp[k] = y_ # Save target for each year in dictionary
    return X_temp,y_temp

X_train, y_train = split(dates_df_train)
X_test, y_test = split_test(dates_df_test_dict)

We are now done with data manipulation, and ready to run the model

In [ ]:
# If model is not run from beginning, but pickled data is loaded in, run this to assign varaibles
input_size = 12
n_features = 31

In [ ]:
# Create RNN model that we use in project
def create_model_rnn(neurons,learning_rate,drop_out,hidden):
    early = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=50) # Stop if no progress in # epochs
    #checkpoint = tf.keras.callbacks.ModelCheckpoint("weights.best.hdf5",monitor='val_loss', verbose=0, save_best_only=True, mode='min')
    callbacks_list = [early] #,checkpoint
    model = Sequential()
    model.add(SimpleRNN(units = neurons, activation = 'relu', input_shape=(input_size, n_features), return_sequences=True))
    model.add(Dropout(rate=drop_out))
    
    # Create functionality for using different amounts of hidden layers used in grid search
    if hidden == 1:
        model.add(SimpleRNN(units = neurons, activation = 'relu', input_shape=(input_size, n_features), return_sequences=True))
    elif  hidden == 2:
        model.add(SimpleRNN(units = neurons, activation = 'relu', input_shape=(input_size, n_features), return_sequences=True))
        model.add(SimpleRNN(units = neurons, activation = 'relu', input_shape=(input_size, n_features), return_sequences=True))
    elif  hidden == 3:
        model.add(SimpleRNN(units = neurons, activation = 'relu', input_shape=(input_size, n_features), return_sequences=True))
        model.add(SimpleRNN(units = neurons, activation = 'relu', input_shape=(input_size, n_features), return_sequences=True))
        model.add(SimpleRNN(units = neurons, activation = 'relu', input_shape=(input_size, n_features), return_sequences=True))
    elif  hidden == 4:
        model.add(SimpleRNN(units = neurons, activation = 'relu', input_shape=(input_size, n_features), return_sequences=True))
        model.add(SimpleRNN(units = neurons, activation = 'relu', input_shape=(input_size, n_features), return_sequences=True))
        model.add(SimpleRNN(units = neurons, activation = 'relu', input_shape=(input_size, n_features), return_sequences=True))
        model.add(SimpleRNN(units = neurons, activation = 'relu', input_shape=(input_size, n_features), return_sequences=True))
        
    model.add(SimpleRNN(units = neurons, activation = 'relu', return_sequences=False ))
    model.add(Dense(units = 1)) #Linear output layer
    opt = optimizers.Adam(lr=learning_rate, clipnorm=1.)  
    model.compile(optimizer = opt, loss = "mse")
    return model,callbacks_list

In [ ]:
#Hyperparameters
neurons = [50]
epochs = [50]
learning_rate = [0.0005,0.001] 
drop_out = [0.05,0.01]
batch_size = [32,64]
hidden = [1,2]
val_size = 0.2
loss = "mse"

Below, we do the grid search. We run a model for every combination of hyperparameters given above and sort the different models on their validation mse.

Bottom function makes it possible to run big grid on firstt month and then use optimal hyperparameters on next 2 months.

In [ ]:
def find_optimal_model(X_train,y_train,starting_point,end_point,epochs,neurons,learning_rate,batch_size,hidden,drop_out):
    optimal = []
    if len(neurons) == 1 & len(epochs) == 1 & len(learning_rate) == 1 & len(drop_out) == 1 & len(batch_size) == 1 & starting_point == end_point:
        verbose = 1
    else:
        verbose = 0
    for i in tqdm_notebook(range(starting_point,end_point+1)):
        print(i)
        print("---")
        temp = []
        for j in range(len(neurons)):
            for q in range(len(epochs)):
                for k in range(len(learning_rate)):
                    for p in range(len(drop_out)):
                        for l in range(len(batch_size)):
                            for h in range(len(hidden)):
                                print("Neurons:",neurons[j],". Epochs:",epochs[q],". Learning rate:",learning_rate[k],". Dropout:",drop_out[p],". Batch size:",batch_size[l], ". Hidden layers:", hidden[h])
                                # Create model using function above
                                model,callbacks_list = create_model_rnn(neurons[j],learning_rate[k],drop_out[p],hidden[h])
                                # Fit model on training data, save loss history
                                hist = model.fit(np.array(X_train[i]),np.array(y_train[i]),epochs=epochs[q], verbose=verbose,callbacks=callbacks_list,validation_split=0.2,batch_size = batch_size[l]) 
                                # Predict data using train as input to calculate train MSE
                                pred_train = model.predict(np.array(X_train[i]))
                                # Predict data using validation data as input to calculate validation MSE
                                pred_val = model.predict(np.array(X_train[i])[int(len(np.array(X_train[i]))*0.8):])
                                # Save dataframe of different MSEs, hyperparameters used, models, and loss results
                                temp.append([mean_squared_error(np.array(y_train[i]),pred_train),mean_squared_error(np.array(y_train[i])[int(len(y_train[i])*0.8):],pred_val),neurons[j],epochs[q],learning_rate[k],drop_out[p],batch_size[l],hidden[h],model,hist.history])
        df_temp = pd.DataFrame(data = temp,columns = ("train mse","val mse","neurons","epochs","learning_rate","dropout","batch size","hidden layers","model_save","loss_hist"))
        optimal.append(df_temp)
    return optimal, starting_point

def run_multiple(X_train,y_train,starting_point,end_point,epochs,neurons,learning_rate,batch_size,hidden,drop_out):
    optimal_1, index_start = find_optimal_model(X_train,y_train,starting_point,starting_point,epochs,neurons,learning_rate,batch_size,hidden,drop_out)
    temp_hyper = optimal_1[0][optimal_1[0]["val mse"]==(optimal_1[0]["val mse"].min())]
    optimal_2, start = find_optimal_model(X_train,y_train,starting_point+1,end_point,[temp_hyper.iloc[0]["epochs"]],[temp_hyper.iloc[0]["neurons"]],[temp_hyper.iloc[0]["learning_rate"]],[temp_hyper.iloc[0]["batch size"]],[temp_hyper.iloc[0]["hidden layers"]],[temp_hyper.iloc[0]["dropout"]])
    return optimal_1+optimal_2,index_start, end_point

In [ ]:
# Run above functions to find models, specify start and end index
start = 0
ending = 6 
optimal1,index_start,index_end = run_multiple(X_train,y_train,start,start+2,epochs,neurons,learning_rate,batch_size,hidden,drop_out)
optimal2,index_start,slut = run_multiple(X_train,y_train,index_end+1,ending,epochs,neurons,learning_rate,batch_size,hidden,drop_out)
optimal = optimal1+optimal2

In [ ]:
# Find every optimal model and save in dataframe along with hyperparameters used
def choose_optimal(optimal,index_start):
    df = pd.DataFrame()
    year = int(np.floor(index_start/12))
    month = index_start % 12
    for i in range(len(optimal)):
        temp = optimal[i][optimal[i]["val mse"]==(optimal[i]["val mse"].min())]
        df = df.append(temp)
        print(year,month)
        month = month+1
    df = df.reset_index(drop=True)
    return df

optimal_df = choose_optimal(optimal,start)

In [ ]:
def choose_optimal_and_predict(optimal,X_test,index_start):
    df = pd.DataFrame()
    year = int(np.floor(index_start/12))
    month = index_start % 12
    
    for i in range(len(optimal)):
        temp = optimal[i][optimal[i]["val mse"]==(optimal[i]["val mse"].min())]
        
#         if i %12 == 0:
#             year = year+1
#             month = 0 #reset month if we change year
        print(year,month)
        temp_pred = temp.iloc[0][8].predict(np.array(X_test[year][month])) #[0][8] chooses the model in this line.
        temp["pred"] = [temp_pred]
        df = df.append(temp)
        month = month+1
    df = df.reset_index(drop=True)
    return df

optimal_df = choose_optimal_and_predict(optimal,X_test,start)